In [1]:
import cpp_soft_info

# Load Data

In [2]:
%reload_ext autoreload
%autoreload 2

In [25]:
from result_saver import SaverProvider
from qiskit_ibm_provider import IBMProvider
# provider = SaverProvider()
provider = IBMProvider()

In [34]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'
DISTANCE = 3
ROUNDS = 3
_RESETS = True



md = metadata_loader(_extract=True, _drop_inutile=False)
# print(md.head())
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 10
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]


md = md[md["meas_level"] == 1]
md = md[1:3]

d = max_distance
synd_rounds = max_distance

md

,creation_date,notebook_name,backend_name,job_id,job_name,job_metadata,tags,meas_level,init_qubits,meas_return,...,job_status,execution_date,extra,optimization_level,code,distance,rounds,logical,layout,descr
242,2023-10-30 09:43:20.984078+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpsa5vayrg008cpk30,NaN,{},[],1.0,True,single,...,JobStatus.DONE,2023-10-31 17:04:50.381134+01:00,NaN,NaN,RepetitionCodeCircuit,10.0,10,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...
235,2023-10-30 09:42:31.121190+01:00,bigger_rep_codes,ibm_sherbrooke,cmzprxv3r3vg008wg7w0,NaN,{},[],1.0,True,single,...,JobStatus.DONE,2023-10-31 17:05:53.332697+01:00,NaN,NaN,RepetitionCodeCircuit,10.0,10,1,_is_hex=True,Run bigger Repetition codes v4: new distances ...


# Initialize the code

In [69]:
import stim
import pymatching

from soft_info import get_repcode_layout, get_repcode_IQ_map
from Scratch import create_or_load_kde_grid


layout = get_repcode_layout(distance=DISTANCE, backend=provider.get_backend(DEVICE), _is_hex=True)
qubit_mapping = get_repcode_IQ_map(layout, synd_rounds) #Hardcoded for repetition codes
print("layout:", layout)
print("len(layout):", len(layout), " = distance * code qubits + (distance-1) * link qubits")

layout: [114, 116, 113, 115, 117]
len(layout): 5  = distance * code qubits + (distance-1) * link qubits


In [31]:
other_date = None # if none then it will find the closest to the tobecalib_job date
# other_date = "2023-11-10T10:30:00" # "2023-11-22T" works to
grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, job_id, 300, 2, other_date=other_date)

# takes 10s

Specified job execution date: 2023-10-31 17:04:50.381134+01:00
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_2pts_3std


# Test backend.properties

In [26]:
backend = provider.get_backend("ibm_sherbrooke")

properties = backend.properties(datetime=None)

In [63]:
print(properties.readout_error(1))
print(properties.readout_length(1))


print(properties.gate_error("sx", 0))
print(properties.gate_error("id", 0))
print(properties.gate_error("x", 0))

print(properties.qubit_property(0))



0.007500000000000062
1.2444444444444443e-06
0.00018511370937386977
0.00018511370937386977
0.00018511370937386977


{'T1': (0.00045493532791432667,
  datetime.datetime(2024, 1, 12, 15, 47, 51, tzinfo=tzlocal())),
 'T2': (0.00028786011204372567,
  datetime.datetime(2024, 1, 11, 21, 18, 40, tzinfo=tzlocal())),
 'frequency': (4635665623.377279,
  datetime.datetime(2024, 1, 13, 11, 3, 32, tzinfo=tzlocal())),
 'anharmonicity': (-313276039.4092362,
  datetime.datetime(2024, 1, 13, 11, 3, 32, tzinfo=tzlocal())),
 'readout_error': (0.0044999999999999485,
  datetime.datetime(2024, 1, 12, 15, 31, 33, tzinfo=tzlocal())),
 'prob_meas0_prep1': (0.0064,
  datetime.datetime(2024, 1, 12, 15, 31, 33, tzinfo=tzlocal())),
 'prob_meas1_prep0': (0.0026000000000000467,
  datetime.datetime(2024, 1, 12, 15, 31, 33, tzinfo=tzlocal())),
 'readout_length': (1.2444444444444443e-06,
  datetime.datetime(2024, 1, 12, 15, 31, 33, tzinfo=tzlocal()))}

# Get round time 

In [185]:
from qiskit import transpile
from qiskit_qec.circuits import RepetitionCodeCircuit


synd_rounds = 1
distance = 3
layout = get_repcode_layout(distance, backend, _is_hex=True)

code = RepetitionCodeCircuit(distance, synd_rounds , xbasis=False, resets=_RESETS)
circ = code.circuit["0"]

transpiled_qc = transpile(circ, backend, initial_layout=layout, optimization_level=3)


13:16:28 Warning: divide by zero encountered in det. IN FILE: /Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/numpy/linalg/linalg.py, LINE: 2180
13:16:28 Warning: invalid value encountered in det. IN FILE: /Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/numpy/linalg/linalg.py, LINE: 2180


In [186]:
# lets get the mean ecr time for the backend

ecr_times = []
for pair in backend.coupling_map:
    ecr_times.append(properties.gate_length('ecr', pair))

print("mean ecr time:", np.mean(ecr_times))
print(ecr_times)

mean ecr time: 5.394074074074071e-07
[5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 8.035555555555555e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-07, 5.333333333333332e-

In [187]:
# transpiled_qc.draw(fold=-1)
print(transpiled_qc.depth())
print(transpiled_qc.duration)

circuit_depth_per_round = transpiled_qc.depth()/synd_rounds
estimated_runtime = circuit_depth_per_round * np.mean(ecr_times)

print("Estimated runtime (in seconds):", estimated_runtime)
print(1000*1e-9)

14
None
Estimated runtime (in seconds): 7.551703703703699e-06
1.0000000000000002e-06


In [188]:
transpiled_qc.draw(fold=-1)

global phase: π/2
                                                                                                                                                                              
     ancilla_0 -> 0 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
     ancilla_1 -> 1 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
     ancilla_2 -> 2 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
     ancilla_3 -> 3 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
     ancilla_4 -> 4 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
     ancilla_5 -> 5 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
     ancilla_6 -> 6 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
     ancilla_7 -> 7 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
     ancilla_8 -> 8 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
     ancilla_9 -> 9 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                              
   ancilla_10 -> 10 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                    

#### Noise model stim

In [59]:
from datetime import datetime

def get_avgs_from_dict(noise_dict):
    '''
    Takes noise dictionary as an argument then returns length 5
    array defining average noise levels: [init, idle, RO, single-, two-qubit gate]
    '''
    init_avg = np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    idle_avg = np.average([noise_dict[qubit]['idle'] for qubit in noise_dict])
    RO_avg = np.average([noise_dict[qubit]['RO'] for qubit in noise_dict])
    single_avg = 1e-4#np.average([noise_dict[qubit]['init'] for qubit in noise_dict])
    two_gate_avg = np.average([noise_dict[qubit]['2-gate'][connection]
                        for qubit in noise_dict
                        for connection in noise_dict[qubit]['2-gate']
                        if connection != 'default'])
    return [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]
# End get_avgs_from_dict


def get_noise_dict_from_backend(backend, layout, date=None):
    '''
    Takes a given backend and a mapping object and returns a noise dictionary
    to pass to a new Cross_Platform_Code/Hex_Code to have accurate backend
    specific noise information based on calibration data.
    '''
    # Pre-reqs
    noise_dict = {}
    round_time = 1000e-9
    if date is None:
        t = None
    else:
        t = datetime(day=int(date[-2:]), month=int(date[-4:-2]), year=int(date[:4]))
    properties = backend.properties(datetime=date)
    # Initializing each qubit with single qubit noise
    for qubit in layout:
        # Defining ROI error
        ROI_error = properties.readout_error(qubit)
        # Defining idle error
        t1 = properties.t1(qubit)
        t2 = properties.t2(qubit)
        t_time = min(t1, t2)
        idle_error = 1 - np.exp(-round_time / t_time)
        # Updating dictionary
        noise_dict[qubit] = {}
        noise_dict[qubit]['init'] = ROI_error
        noise_dict[qubit]['idle'] = idle_error
        noise_dict[qubit]['RO'] = ROI_error
        noise_dict[qubit]['gate'] = 1e-4
        noise_dict[qubit]['2-gate'] = {'default': 1e-2}
    # Two qubit gate noise
    all_qiskit_indexes = layout 
    for pair in backend.coupling_map:
        # Making sure connection is in circuit
        if pair[0] in all_qiskit_indexes and pair[1] in all_qiskit_indexes:
            # Getting two qubit gate error
            two_gate_error = properties.gate_error('ecr', pair)
            if two_gate_error > .5:
                two_gate_error = .5
            # # Updating dictionary
            # stim_index_0 = map.get_stim_index(qiskit_index=pair[0])
            # stim_index_1 = map.get_stim_index(qiskit_index=pair[1])
            noise_dict[pair[0]]['2-gate'][pair[1]] = two_gate_error
            noise_dict[pair[1]]['2-gate'][pair[0]] = two_gate_error
    return noise_dict
# End get_noise_dict_from_backend

In [60]:

date = "2023-10-30 09:45:17.340386+01:00" #md.creation_date[0] # md[0] for log 1
noise_dict = get_noise_dict_from_backend(provider.get_backend(DEVICE), layout, date=None) # takes noise closest to today
print("noise_dict:", noise_dict)
avgs_noise = get_avgs_from_dict(noise_dict)
print("avgs_noise [init_avg, idle_avg, RO_avg, single_avg, two_gate_avg]:", avgs_noise)

# takes 4s

noise_dict: {114: {'init': 0.010800000000000032, 'idle': 0.004715134260089182, 'RO': 0.010800000000000032, 'gate': 0.0001, '2-gate': {'default': 0.01, 113: 0.014721939501277842, 115: 0.022587926276817327}}, 116: {'init': 0.01330000000000009, 'idle': 0.005265778063089477, 'RO': 0.01330000000000009, 'gate': 0.0001, '2-gate': {'default': 0.01, 115: 0.004367094280224915, 117: 0.009455025576278975}}, 118: {'init': 0.0040999999999999925, 'idle': 0.005581224104389326, 'RO': 0.0040999999999999925, 'gate': 0.0001, '2-gate': {'default': 0.01, 117: 0.007327570066070099, 119: 0.009963833140667261}}, 120: {'init': 0.007000000000000006, 'idle': 0.005101770918414905, 'RO': 0.007000000000000006, 'gate': 0.0001, '2-gate': {'default': 0.01, 119: 0.02178150644472393, 121: 0.007173375924125536}}, 122: {'init': 0.005099999999999993, 'idle': 0.003119260580129568, 'RO': 0.005099999999999993, 'gate': 0.0001, '2-gate': {'default': 0.01, 121: 0.00769794866438675, 111: 0.00842212142256385}}, 104: {'init': 0.0030

In [61]:
from soft_info import draw_matching_graph



circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=d,
                                rounds=synd_rounds,
                                after_clifford_depolarization= avgs_noise[4], #two-qubit-fidelity,
                                after_reset_flip_probability= 0,# avgs_noise[0], #reset error,
                                before_measure_flip_probability= avgs_noise[2]*5, #measurement error,
                                before_round_data_depolarization= avgs_noise[1]) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

print(circuit)

R 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18
TICK
DEPOLARIZE1(0.00664425) 0 2 4 6 8 10 12 14 16 18
CX 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
DEPOLARIZE2(0.00929879) 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
TICK
CX 2 1 4 3 6 5 8 7 10 9 12 11 14 13 16 15 18 17
DEPOLARIZE2(0.00929879) 2 1 4 3 6 5 8 7 10 9 12 11 14 13 16 15 18 17
TICK
X_ERROR(0.0599737) 1 3 5 7 9 11 13 15 17
MR 1 3 5 7 9 11 13 15 17
DETECTOR(1, 0) rec[-9]
DETECTOR(3, 0) rec[-8]
DETECTOR(5, 0) rec[-7]
DETECTOR(7, 0) rec[-6]
DETECTOR(9, 0) rec[-5]
DETECTOR(11, 0) rec[-4]
DETECTOR(13, 0) rec[-3]
DETECTOR(15, 0) rec[-2]
DETECTOR(17, 0) rec[-1]
REPEAT 9 {
    TICK
    DEPOLARIZE1(0.00664425) 0 2 4 6 8 10 12 14 16 18
    CX 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
    DEPOLARIZE2(0.00929879) 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
    TICK
    CX 2 1 4 3 6 5 8 7 10 9 12 11 14 13 16 15 18 17
    DEPOLARIZE2(0.00929879) 2 1 4 3 6 5 8 7 10 9 12 11 14 13 16 15 18 17
    TICK
    X_ERROR(0.0599737) 1 3 5 7 9 11 13 